# Question 2

In this question we will:
- Implement the vector median filter for colour images.
- Apply the vector median filter to a small dataset and establish a baseline performance meteric.
- Try to design a denoising pipeline that outperforms using only the vector median filter.   

In [4]:
# Boilerplate imports
import skimage.io as skio
import skimage.color as skcolor
import skimage.util as skutil
import skimage.restoration as skrest
import skimage.filters as skfilt
import skimage.metrics as skmetrics
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

np.random.seed(0)

# Step 1: Implement the vector median filter for colour images.

## Instructions

- Write a function that implements the vector median filter.  Details on the vector median filter can be found in Topic 4 of the lecture notes and implementation suggestions can be found in Section 2 of the asignment PDF.  
- The parameters to your function should be the input image `I`, and the `radius` of the square neighbourhood, e.g. radius 1 = 3x3 filter, radius 2 = 5x5 filter, etc.
- Your function should accept images of dtype `uint8` or `float64` (other types may be rejected if you wish).    It is easiest if you work with the image as data type `float64`.   If the input image is of type `uint8`, it is suggested that you convert it to dtype `float64`, construct the output image also as type `float64` and then convert it back to `uint8` before returning it.  In any case, the output image returned should be the same dtype as the input image.

In [70]:
def vector_median(I, radius):
    # Type Check
    if I.dtype not in [np.uint8, np.float64]:
        raise ValueError("Image type not supported. Must be uint8 or float64")

    # Valid radius
    if radius < 1:
        raise ValueError("Radius must be greater than or equal to 1")

    # If uint8, convert to float64
    is_uint8 = I.dtype == np.uint8
    if is_uint8:
        I = skutil.img_as_float(I)

    # Get dimensions
    height, width, channels = I.shape
    output = np.zeros_like(I)  # Stores the output image

    # Add padding to the image to help with the borders
    I_padded = np.pad(
        I,  # Image to pad
        (
            (radius, radius),  # Pad top and bottom 
            (radius, radius), # Pad left and right
            (0, 0)  # Pad channels
        ),
        mode='edge'  # Pad with the nearest pixel
    )
    
    # Iterate over the image
    for y in range(height):
        for x in range(width):
            # Adjust x, y to start on the image and not the padding
            x_pad = x + radius
            y_pad = y + radius

            # Get the neighbourhood
            neighbourhood = I_padded[
                                y_pad - radius:y_pad + radius + 1,  # Add 1 bc slicing is exclusive
                                x_pad - radius:x_pad + radius + 1, 
                                :
                            ]

            # Compute the median for each channel
            median = np.median(neighbourhood, axis=(0, 1))

            # Find the closest pixel to the median using Manhattan distance
            distances = np.sum(np.abs(neighbourhood - median), axis=2)  # Compute the Manhattan distance for all pixels in the neighbourhood
            min_index = np.unravel_index(  # Unravel the index to get the x, y coordinates (returns a tuple)
                np.argmin(distances),  # Find the index that points to the minimum distance
                distances.shape 
            )
            closest_pixel = neighbourhood[min_index]

            # Assign the closest pixel to the output
            output[y, x, :] = closest_pixel

    if is_uint8:  # Parse back to uint8
        return skutil.img_as_ubyte(output)
    else:
        return output

# Step 2a: Establish a Baseline

## Instructions

- Determine the PSNR and SSIM of all of the noisy images by comparing them to their noiseless versions -- store these in a list.  
    - Use the same method to read the images files as in question 1, but read them from the `images/noisy` directory and the `images/noiseless` directory as needed.
- Filter each noisy image with a 3x3 vector median filter and compute the PSNR and SSIM of all of the filtered noisy images (by comparing them to the noiseless images) -- store these in a list.   
    - As with question 1, to save time we recommend you develop and debug your programs using only the first three to five images in file_list, and run it on the complete list when you're done. 
- The noiseless images are provided so you can compute PSNR and SSIM.  See functions `skmetrics.peak_signal_noise_ratio()` and `skmetrics.structural_similarity()`.  You'll need the `channel_axis=2` argument for `structural_similarity()`.  This indicates to the function that the it is the 3rd dimension of the 3D arrays that index the R, G and B channels.

_Hint: This step requires doing a few seemingly different things, but it's actually easier to do them all at once in the same loop._



In [71]:
file_list = pd.read_csv('data/images.csv', header=None)
file_list = file_list[0].tolist()

['100075.png', '100080.png']

In [72]:
noiseless_dir = 'data/images/noiseless'
noisy_dir = 'data/images/noisy'

In [73]:
for filename in file_list[:3]:   # you can restrict this to, for example,  file_list[0:3], for testing
    noiseless_path = os.path.join(noiseless_dir, filename)
    noisy_path = os.path.join(noisy_dir, filename)
    
    # Read the images
    noiseless = skio.imread(noiseless_path)
    noiseless = skutil.img_as_float(noiseless)
    
    noisy = skio.imread(noisy_path)
    noisy = skutil.img_as_float(noisy)
    
    # Compute PSNR and SSIM for the noiseless image
    psnr = skmetrics.peak_signal_noise_ratio(noiseless, noisy)
    # Ensure window size does not exceed image size
    ssim = skmetrics.structural_similarity(noiseless, noisy, data_range=1, channel_axis=2)
    
    # Filter the noisy image
    filtered = vector_median(noisy, 1)
    
    # Compute PSNR and SSIM for the filtered image
    psnr_filtered = skmetrics.peak_signal_noise_ratio(noiseless, filtered)
    ssim_filtered = skmetrics.structural_similarity(noiseless, filtered, data_range=1, channel_axis=2)

(np.int64(0), np.int64(0))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(0))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(0))
(np.int64(2), np.int64(2))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(0))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(0))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(1))
(np.int64(2), np.int64(2))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(0))
(np.int64(0), np.int64(0))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(0))
(np.int64(0), np.int64(2))
(np.int64(0), np.int64(0))
(np.int64(2), np.int64(0))
(np.int64(0), np.int64(1))
(np.int64(0), np.int64(0))
(

KeyboardInterrupt: 

# Step 2b: Compute average baseline metrics.

## Instructions

Compute the average PSNR and SSIM for the unfiltered noisy images (take the PSNR and SSIM measures for each image and average them).  Do the same for the median-filtered noisy images.  Plot a bar graph showing the two mean PSNR values and another showing the two mean SSIM values before and after filtering (see assignment description for sample output).

In [3]:
# some modules you might need.

import matplotlib.pyplot as plt
import numpy as np

# Step 3:  See if you can do better!

Find a better way to denoise the images.  Compute the mean PSNR and SSIM for your denoising method.   Replot the graphs in the previous step with the new results added.

_You can use any functions in the python libraries to help you (numpy, scipy, skimage, etc.) but you cannot bring in code from external sources._


In [4]:
# Write your de-noising method here.